# Day17_0: CNN (합성곱 신경망) - 정답

## 📚 실습 퀴즈 정답

---

In [ ]:
# 공통 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

---

### Q1. Conv2d 출력 크기 계산 ⭐

**문제**: 64x64 크기의 RGB 이미지에 다음 Conv2d를 적용했을 때 출력 크기를 계산하세요.

```python
nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2)
```

In [ ]:
# 정답 코드

# 출력 크기 계산 공식: (입력 + 2*padding - kernel) / stride + 1
def calc_output_size(input_size, kernel_size, stride, padding):
    return (input_size + 2 * padding - kernel_size) // stride + 1

# 주어진 조건
input_size = 64
kernel_size = 5
stride = 2
padding = 2

# 계산
output_size = calc_output_size(input_size, kernel_size, stride, padding)
print(f"계산: ({input_size} + 2*{padding} - {kernel_size}) / {stride} + 1 = {output_size}")

# 실제 확인
conv = nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2)
dummy_input = torch.randn(1, 3, 64, 64)
output = conv(dummy_input)
print(f"\n실제 출력: {output.shape}")
print(f"출력 shape: (1, 32, {output_size}, {output_size})")

### 💡 풀이 설명

**접근 방법**:
- Conv2d 출력 크기 공식 적용: (입력 + 2*padding - kernel) / stride + 1

**핵심 개념**:
- `kernel_size=5`: 5x5 크기 필터
- `stride=2`: 2칸씩 이동 (출력 크기 절반)
- `padding=2`: 가장자리에 2칸 패딩 추가

**계산 과정**:
- (64 + 4 - 5) / 2 + 1 = 63 / 2 + 1 = 31 + 1 = 32

**실무 팁**:
- `padding = (kernel_size - 1) / 2`일 때 출력 크기가 입력/stride와 같아짐
- stride=2는 흔한 다운샘플링 방법

---

### Q2. Max Pooling 효과 ⭐

**문제**: 4x4 특징 맵에 2x2 Max Pooling (stride=2)을 적용한 결과를 출력하세요.

In [ ]:
# 정답 코드

# 입력 특징 맵
feature_map = torch.tensor([[1, 3, 2, 4],
                            [5, 6, 7, 8],
                            [3, 2, 1, 0],
                            [9, 8, 7, 6]], dtype=torch.float32)

# (배치, 채널) 차원 추가
feature_map_4d = feature_map.unsqueeze(0).unsqueeze(0)  # (1, 1, 4, 4)

# Max Pooling 적용
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
output = max_pool(feature_map_4d)

print("입력 특징 맵:")
print(feature_map)
print(f"\n입력 shape: {feature_map.shape}")

print("\n2x2 Max Pooling 결과:")
print(output.squeeze())
print(f"\n출력 shape: {output.squeeze().shape}")

### 💡 풀이 설명

**접근 방법**:
- 2x2 영역에서 최댓값을 선택
- stride=2로 겹치지 않게 이동

**핵심 개념**:
- 좌상단 2x2 영역 [1,3,5,6] -> max = 6
- 우상단 2x2 영역 [2,4,7,8] -> max = 8
- 좌하단 2x2 영역 [3,2,9,8] -> max = 9
- 우하단 2x2 영역 [1,0,7,6] -> max = 7

**결과**: [[6, 8], [9, 7]]

**실무 팁**:
- Max Pooling은 가장 강한 특징만 남김
- Average Pooling보다 특징 보존에 유리

---

### Q3. 이미지 정규화 ⭐⭐

**문제**: 0-255 범위의 이미지를 [-1, 1] 범위로 정규화하는 코드를 작성하세요.

In [ ]:
# 정답 코드

# 0-255 범위 이미지 (시뮬레이션)
raw_image = torch.randint(0, 256, (3, 32, 32), dtype=torch.float32)

# 방법 1: 수동 계산
# 먼저 [0, 1]로 변환 후 [-1, 1]로 조정
normalized_image = (raw_image / 255.0 - 0.5) / 0.5

# 또는 한 번에: (x - 127.5) / 127.5
normalized_image_v2 = (raw_image - 127.5) / 127.5

print(f"원본 범위: [{raw_image.min():.0f}, {raw_image.max():.0f}]")
print(f"정규화 후 범위: [{normalized_image.min():.4f}, {normalized_image.max():.4f}]")

# 검증
print(f"\n0 -> {(0 / 255.0 - 0.5) / 0.5:.4f}")
print(f"127.5 -> {(127.5 / 255.0 - 0.5) / 0.5:.4f}")
print(f"255 -> {(255 / 255.0 - 0.5) / 0.5:.4f}")

### 💡 풀이 설명

**접근 방법**:
- 2단계: [0, 255] -> [0, 1] -> [-1, 1]
- 또는 직접: (x - 127.5) / 127.5

**핵심 개념**:
- `/255`: [0, 1] 범위로 변환
- `-0.5`: [−0.5, 0.5] 범위로 이동
- `/0.5`: [-1, 1] 범위로 스케일

**대안 - transforms 사용**:
```python
transforms.Normalize((0.5,), (0.5,))  # 이미 [0,1]인 텐서에 적용
```

**실무 팁**:
- ToTensor()는 자동으로 [0, 255] -> [0, 1] 변환
- Normalize()는 (x - mean) / std 적용

---

### Q4. CNN 아키텍처 분석 ⭐⭐

**문제**: 다음 CNN 모델의 각 레이어 출력 shape를 계산하세요.

In [ ]:
# 정답 코드

model = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1),    # (1, 16, 32, 32)
    nn.ReLU(),
    nn.MaxPool2d(2, 2),                 # (1, 16, 16, 16)
    nn.Conv2d(16, 32, 3, padding=1),   # (1, 32, 16, 16)
    nn.ReLU(),
    nn.MaxPool2d(2, 2),                 # (1, 32, 8, 8)
    nn.Flatten(),                       # (1, 2048)
)

# 더미 입력으로 실제 shape 확인
x = torch.randn(1, 3, 32, 32)
print(f"입력: {x.shape}")
print("-" * 40)

for i, layer in enumerate(model):
    x = layer(x)
    print(f"{layer.__class__.__name__}: {x.shape}")

### 💡 풀이 설명

**접근 방법**:
- 각 레이어별로 출력 shape 추적

**핵심 개념**:
1. Conv2d(3, 16, 3, padding=1): 채널 3->16, 크기 유지 (padding=1)
2. ReLU: shape 변화 없음
3. MaxPool2d(2,2): 공간 크기 절반 (32->16)
4. Conv2d(16, 32, 3, padding=1): 채널 16->32, 크기 유지
5. MaxPool2d(2,2): 공간 크기 절반 (16->8)
6. Flatten: (1, 32, 8, 8) -> (1, 2048)

**실무 팁**:
- 디버깅 시 중간 출력 shape 확인 필수
- Flatten 후 Linear 입력 크기 = C * H * W

---

### Q5. Data Augmentation 적용 ⭐⭐

**문제**: 다양한 증강 기법을 포함하는 transform 파이프라인을 작성하세요.

In [ ]:
# 정답 코드

train_transform = transforms.Compose([
    # 1. 무작위 크롭 (32x32, padding=4)
    transforms.RandomCrop(32, padding=4),
    
    # 2. 무작위 수평 뒤집기
    transforms.RandomHorizontalFlip(),
    
    # 3. 무작위 회전 (최대 15도)
    transforms.RandomRotation(15),
    
    # 4. 텐서 변환
    transforms.ToTensor(),
    
    # 5. 정규화 (mean=0.5, std=0.5)
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

print("Data Augmentation 파이프라인:")
print(train_transform)

# 테스트
from PIL import Image
dummy_pil = Image.new('RGB', (32, 32), color='red')
augmented = train_transform(dummy_pil)
print(f"\n증강 후 shape: {augmented.shape}")
print(f"값 범위: [{augmented.min():.4f}, {augmented.max():.4f}]")

### 💡 풀이 설명

**접근 방법**:
- transforms.Compose로 여러 변환 연결
- 순서 중요: 기하 변환 -> ToTensor -> 정규화

**핵심 개념**:
- RandomCrop(32, padding=4): 먼저 40x40으로 패딩 후 32x32 크롭
- RandomHorizontalFlip(): 기본 50% 확률
- RandomRotation(15): -15 ~ +15도 회전

**대안**:
```python
transforms.ColorJitter(brightness=0.2)  # 밝기 변환
transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # 이동
```

**실무 팁**:
- ToTensor()는 PIL Image 입력 필요
- Normalize는 ToTensor 이후에 적용

---

### Q6. torchvision 데이터셋 로드 ⭐⭐

**문제**: MNIST 데이터셋을 로드하고, 첫 번째 이미지의 shape와 레이블을 출력하세요.

In [ ]:
# 정답 코드

# MNIST 데이터셋 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 통계
])

mnist_dataset = torchvision.datasets.MNIST(
    root='./datasets',
    train=True,
    download=True,
    transform=transform
)

# 첫 번째 이미지 확인
image, label = mnist_dataset[0]

print(f"데이터셋 크기: {len(mnist_dataset)}")
print(f"이미지 shape: {image.shape}")
print(f"레이블: {label}")
print(f"픽셀 값 범위: [{image.min():.4f}, {image.max():.4f}]")

### 💡 풀이 설명

**접근 방법**:
- torchvision.datasets 사용
- download=True로 자동 다운로드

**핵심 개념**:
- MNIST: 28x28 흑백 손글씨 숫자 (0-9)
- shape: (1, 28, 28) - 1채널, 28x28
- 60,000개 훈련 이미지, 10,000개 테스트

**MNIST vs CIFAR-10**:
| 데이터셋 | 이미지 크기 | 채널 | 클래스 |
|---------|-----------|------|--------|
| MNIST | 28x28 | 1 | 10 (숫자) |
| CIFAR-10 | 32x32 | 3 | 10 (객체) |

**실무 팁**:
- 정규화 값은 데이터셋별로 다름
- MNIST: mean=0.1307, std=0.3081

---

### Q7. Transfer Learning 설정 ⭐⭐⭐

**문제**: 사전 학습된 ResNet18을 5개 클래스 분류에 적용하세요.

In [ ]:
# 정답 코드

# 1. 사전 학습된 ResNet18 로드
model = models.resnet18(weights='IMAGENET1K_V1')

# 2. 모든 파라미터 동결
for param in model.parameters():
    param.requires_grad = False

# 3. 마지막 FC 레이어 교체 (5개 클래스용)
num_features = model.fc.in_features  # 512
model.fc = nn.Linear(num_features, 5)

# 학습 가능한 파라미터 확인
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"전체 파라미터: {total_params:,}")
print(f"학습 가능 파라미터: {trainable_params:,}")
print(f"동결된 파라미터: {total_params - trainable_params:,}")
print(f"\n새 FC 레이어: {model.fc}")

In [ ]:
# 테스트 - 학습 가능한 파라미터만 출력
print("학습 가능한 파라미터:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.shape}")

### 💡 풀이 설명

**접근 방법**:
1. 사전 학습 모델 로드
2. 모든 레이어 동결 (requires_grad=False)
3. 마지막 FC 레이어만 새로 정의

**핵심 개념**:
- Feature Extraction: 사전 학습 가중치는 특징 추출기로 사용
- 새 FC 레이어만 학습
- 학습 파라미터: weight(512*5) + bias(5) = 2,565개

**대안 - 더 복잡한 FC**:
```python
model.fc = nn.Sequential(
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 5)
)
```

**실무 팁**:
- 데이터가 매우 적을 때 Feature Extraction 권장
- 새 FC 레이어는 자동으로 requires_grad=True

---

### Q8. Fine-tuning 구현 ⭐⭐⭐

**문제**: ResNet18에서 layer4와 fc 레이어만 학습 가능하도록 설정하세요.

In [ ]:
# 정답 코드

# 1. 사전 학습 모델 로드
model = models.resnet18(weights='IMAGENET1K_V1')

# 2. 모든 파라미터 동결
for param in model.parameters():
    param.requires_grad = False

# 3. layer4 동결 해제
for param in model.layer4.parameters():
    param.requires_grad = True

# 4. FC 레이어 교체 (새 레이어는 자동으로 requires_grad=True)
model.fc = nn.Linear(model.fc.in_features, 10)

# 파라미터 통계
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"전체 파라미터: {total_params:,}")
print(f"학습 가능 파라미터: {trainable_params:,}")
print(f"학습 비율: {trainable_params/total_params:.1%}")

In [ ]:
# 학습 가능한 레이어 확인
print("학습 가능한 레이어:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.numel():,}")

### 💡 풀이 설명

**접근 방법**:
1. 전체 동결
2. layer4 해제
3. FC 교체

**핵심 개념**:
- Fine-tuning: 사전 학습 가중치를 시작점으로 재학습
- 보통 마지막 몇 개 레이어만 학습
- 낮은 학습률 필수 (기존 가중치 파괴 방지)

**ResNet18 구조**:
```
conv1 -> bn1 -> relu -> maxpool
-> layer1 -> layer2 -> layer3 -> layer4
-> avgpool -> fc
```

**실무 팁**:
- 데이터가 어느 정도 있으면 Fine-tuning 추천
- 학습률: Feature Extraction보다 10배 낮게 (1e-4 정도)

---

### Q9. 커스텀 CNN 모델 ⭐⭐⭐⭐

**문제**: 주어진 조건의 CNN 모델을 정의하세요.

In [ ]:
# 정답 코드

class CustomCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # Conv 블록 1: (1, 28, 28) -> (32, 14, 14)
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        # Conv 블록 2: (32, 14, 14) -> (64, 7, 7)
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        # Conv 블록 3: (64, 7, 7) -> (128, 3, 3)
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        # Global Average Pooling: (128, 3, 3) -> (128, 1, 1)
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # 출력 레이어
        self.fc = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# 모델 생성 및 테스트
model = CustomCNN(num_classes=10)
print(model)

# 순전파 테스트
dummy_input = torch.randn(4, 1, 28, 28)
output = model(dummy_input)
print(f"\n입력: {dummy_input.shape}")
print(f"출력: {output.shape}")
print(f"\n총 파라미터: {sum(p.numel() for p in model.parameters()):,}")

### 💡 풀이 설명

**접근 방법**:
- Conv Block: Conv -> BatchNorm -> ReLU -> MaxPool
- GAP로 공간 차원 제거
- 단순한 FC 레이어로 분류

**핵심 개념**:
- BatchNorm: 학습 안정화, 빠른 수렴
- Global Average Pooling: FC 파라미터 대폭 감소
- 채널 수 증가, 공간 크기 감소 패턴

**Shape 추적**:
- 입력: (1, 28, 28)
- Block1 후: (32, 14, 14)
- Block2 후: (64, 7, 7)
- Block3 후: (128, 3, 3)
- GAP 후: (128, 1, 1) = 128
- FC: 128 -> 10

**실무 팁**:
- BatchNorm은 Conv 직후, 활성화 함수 직전에 배치
- GAP 사용 시 FC 입력 크기가 채널 수와 동일

---

### Q10. 전체 학습 파이프라인 ⭐⭐⭐⭐⭐

**문제**: CIFAR-10 데이터셋으로 CNN을 학습하는 전체 파이프라인을 구현하세요.

In [ ]:
# 정답 코드 - Part 1: 데이터 준비

# 1. Data Augmentation 포함 전처리
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# 2. 데이터셋 로드
data_path = './datasets'
train_dataset = torchvision.datasets.CIFAR10(
    root=data_path, train=True, download=True, transform=train_transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root=data_path, train=False, download=True, transform=test_transform
)

# 3. DataLoader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print(f"훈련 데이터: {len(train_dataset)}, 테스트 데이터: {len(test_dataset)}")
print(f"훈련 배치: {len(train_loader)}, 테스트 배치: {len(test_loader)}")

In [ ]:
# 정답 코드 - Part 2: 모델 정의

class CIFAR10CNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            
            # Block 2
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            
            # Block 3
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 모델, 손실 함수, 옵티마이저
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CIFAR10CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Device: {device}")
print(f"파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 정답 코드 - Part 3: 학습 및 평가 함수

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

In [ ]:
# 정답 코드 - Part 4: 학습 실행 (3 에포크)

epochs = 3
history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}

print("="*60)
print("학습 시작!")
print("="*60)

for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
    
    print(f"Epoch {epoch+1}/{epochs}: "
          f"Train Loss={train_loss:.4f}, Acc={train_acc:.2f}% | "
          f"Test Loss={test_loss:.4f}, Acc={test_acc:.2f}%")

print("="*60)
print("학습 완료!")

In [ ]:
# 정답 코드 - Part 5: 학습 곡선 시각화 (Plotly)

# 데이터프레임 생성
history_df = pd.DataFrame(history)
history_df['epoch'] = range(1, epochs + 1)

# 서브플롯 생성
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['손실 (Loss)', '정확도 (Accuracy)']
)

# 손실 곡선
fig.add_trace(
    go.Scatter(x=history_df['epoch'], y=history_df['train_loss'], 
               name='Train Loss', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=history_df['epoch'], y=history_df['test_loss'], 
               name='Test Loss', line=dict(color='red', dash='dash')),
    row=1, col=1
)

# 정확도 곡선
fig.add_trace(
    go.Scatter(x=history_df['epoch'], y=history_df['train_acc'], 
               name='Train Acc', line=dict(color='green')),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=history_df['epoch'], y=history_df['test_acc'], 
               name='Test Acc', line=dict(color='orange', dash='dash')),
    row=1, col=2
)

# 레이아웃 설정
fig.update_layout(
    title='CIFAR-10 CNN 학습 곡선',
    height=400,
    showlegend=True
)
fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_xaxes(title_text='Epoch', row=1, col=2)
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Accuracy (%)', row=1, col=2)

fig.show()

### 💡 풀이 설명

**전체 파이프라인 구조**:
1. **데이터 준비**: Augmentation + 정규화 + DataLoader
2. **모델 정의**: Conv-BN-ReLU-Pool 패턴
3. **학습 함수**: train/eval 모드 분리
4. **학습 실행**: 에포크 루프
5. **시각화**: Plotly로 학습 곡선

**핵심 개념**:
- BatchNorm: 각 Conv 후에 적용
- Dropout: 과적합 방지 (Conv 블록: 0.25, FC: 0.5)
- Adam 옵티마이저: 자동 학습률 조정

**모델 구조**:
```
입력 (3, 32, 32)
-> Block1 (32, 16, 16)
-> Block2 (64, 8, 8)  
-> Block3 (128, 4, 4)
-> Flatten (2048)
-> FC (256) -> FC (10)
```

**추가 개선 방법**:
1. 더 많은 에포크 학습 (50-100)
2. Learning Rate Scheduler 추가
3. 더 강한 Data Augmentation
4. Transfer Learning (ResNet 등)

**실무 팁**:
- 3 에포크는 빠른 테스트용, 실제로는 50+ 에포크 필요
- Early Stopping으로 과적합 방지
- 학습 곡선으로 과적합/과소적합 진단

---

## 📊 학습 정리

### CNN 핵심 개념 요약

| 개념 | 핵심 | 실무 팁 |
|-----|------|--------|
| Conv2d | 지역 특징 추출 | padding=(k-1)/2로 크기 유지 |
| MaxPool | 다운샘플링 | 2x2, stride=2가 표준 |
| BatchNorm | 학습 안정화 | Conv 직후 적용 |
| Dropout | 과적합 방지 | Conv: 0.25, FC: 0.5 |
| GAP | 파라미터 감소 | FC 대신 사용 가능 |

### 전이 학습 전략

| 상황 | 전략 | 설정 |
|-----|------|------|
| 데이터 매우 적음 | Feature Extraction | 전체 동결, FC만 학습 |
| 데이터 적당 | Fine-tuning | 뒷부분 해제, 낮은 LR |
| 데이터 충분 | 전체 재학습 | 사전 학습 가중치로 초기화 |

### 실무 체크리스트

- [ ] Data Augmentation 적용 (훈련 데이터)
- [ ] 적절한 정규화 (데이터셋별 통계)
- [ ] BatchNorm 사용
- [ ] Dropout 적용
- [ ] 학습 곡선 모니터링
- [ ] Early Stopping 고려